In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d
import numpy as np
from astropy.cosmology import LambdaCDM
from astropy.table import Table
from math import atan2, degrees

In [ ]:
#Label line with line2D label data
def labelLine(line,x,label=None,align=True,**kwargs):

    ax = line.axes
    xdata = line.get_xdata()
    ydata = line.get_ydata()

    if (x < xdata[0]) or (x > xdata[-1]):
        print('x label location is outside data range!')
        return

    #Find corresponding y co-ordinate and angle of the line
    ip = 1
    for i in range(len(xdata)):
        if x < xdata[i]:
            ip = i
            break

    y = ydata[ip-1] + (ydata[ip]-ydata[ip-1])*(x-xdata[ip-1])/(xdata[ip]-xdata[ip-1])

    if not label:
        label = line.get_label()

    if align:
        #Compute the slope
        dx = xdata[ip] - xdata[ip-1]
        dy = ydata[ip] - ydata[ip-1]
        ang = degrees(atan2(dy,dx))

        #Transform to screen co-ordinates
        pt = np.array([x,y]).reshape((1,2))
        trans_angle = ax.transData.transform_angles(np.array((ang,)),pt)[0]

    else:
        trans_angle = 0

    #Set a bunch of keyword arguments
    if 'color' not in kwargs:
        kwargs['color'] = line.get_color()

    if ('horizontalalignment' not in kwargs) and ('ha' not in kwargs):
        kwargs['ha'] = 'center'

    if ('verticalalignment' not in kwargs) and ('va' not in kwargs):
        kwargs['va'] = 'center'

    if 'backgroundcolor' not in kwargs:
        kwargs['backgroundcolor'] = ax.get_facecolor()

    if 'clip_on' not in kwargs:
        kwargs['clip_on'] = True

    if 'zorder' not in kwargs:
        kwargs['zorder'] = 2.5

    ax.text(x,y,label,rotation=trans_angle,**kwargs)

def labelLines(lines,align=True,xvals=None,**kwargs):

    ax = lines[0].axes
    labLines = []
    labels = []

    #Take only the lines which have labels other than the default ones
    for line in lines:
        label = line.get_label()
        if "_line" not in label:
            labLines.append(line)
            labels.append(label)

    if xvals is None:
        xmin,xmax = ax.get_xlim()
        xvals = np.linspace(xmin,xmax,len(labLines)+2)[1:-1]

    for line,x,label in zip(labLines,xvals,labels):
        labelLine(line,x,label,align,**kwargs)

In [ ]:
def Lx_calc(mass, z, h):
    '''Compute the Lx for a cluster of a specific mass and at a specific redshift.
    
    This is using the Vikhlininet al. (2009) relationship (see Eq. 22).
    
    Little h comes from the cosmology
    
    '''
    lnLx = 47.392 + 1.61 * np.log(mass * 1e14) + 1.850 * np.log(cosmo.efunc(z)) - 0.39 * np.log(h / 0.72)
    Lx = np.exp(lnLx)
    return Lx

In [ ]:
# read in the catalog
xray_data = Table.read('./external_data/blank_table_nh_fluxc_lumc_uplims.txt', format='ascii')

# only use non-duplicates
# xray_data = xray_data[~xray_data['DUPLICATE']]

# read psz data
psz_data = Table.read('./external_data/psz_xxvii_fig26.csv')

zarr = psz_data['z']
y = psz_data['m500']

In [ ]:
# convert PSZ masses to Lx

# Define the cosmology
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7, Tcmb0=2.725)
h = cosmo.H0.value/100

Lx = Lx_calc(y, zarr, h)

fig, ax = plt.subplots(figsize=(7, 7 * (np.sqrt(5.) - 1.0) / 2.0))
ax.set_xlim(0.02,0.8)
ax.set_ylim(0.001, 100)
ax.tick_params(zorder=10)
ax.semilogy()

# planck limits
ax.plot(zarr, Lx/1e44, color='#A60628', label='PSZ 20% Completeness Limit', zorder=1)

# add planck errors
ax.fill_between(zarr, 0.673 * Lx / 1e44, 1.486 * Lx / 1e44, color='#A60628', alpha=0.3, zorder=1)


maxsize = xray_data['XRT_EXP'].max()/2

# our cluster points
line = ax.scatter(xray_data['REDSHIFT'], xray_data['lumUpLim']/1e44, c='#348ABD', marker='v', 
                  s=(xray_data['XRT_EXP']/maxsize * 500 + 5), label='This Work', zorder=2)

# for fixed mass
for i in range(1,20,2):
    Lx = Lx_calc(i, zarr, h)
    ax.plot(zarr, Lx/1e44, lw=1, c='0.6', zorder=-i)

ax.set_xlabel('Redshift')
ax.set_ylabel('L$_x$ [$10^{44}~erg~s^{-1}$]')

# label the fixed mass lines
lines = ax.get_lines()

for i, j in enumerate(range(1,20,2)):
    if not i % 2:
        labelLine(lines[1+i], 0.4 + 0.05 * i/1.5, f'{j}e14', zorder=-i)

# Custom Legend
legend_elements = [Line2D([0], [0], linewidth=0, marker='v', color='#348ABD', label='This Work'),
                   Line2D([0], [0], linewidth=2, color='#A60628', label='PSZ 20% Completeness Limit'), 
                   Line2D([0], [0], lw=1, color='0.6', label='$L_x$ for Fixed Mass') 
                  ]
ax.legend(handles=legend_elements, loc='lower right')

In [ ]:
plt.tight_layout()

In [ ]:
plt.savefig('./figures/Lx_z_upper.pdf', bbox='tight')

In [ ]:
plt.savefig('./figures/Lx_z_upper.png', bbox='tight')